In [1]:
%cd ..

/home/soda/rcappuzz/work/benchmark-join-suggestions


In [3]:
import json
import os
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import sklearn.metrics as metrics
from joblib import Parallel, delayed, dump, load
from tqdm import tqdm
import seaborn as sns
import polars.selectors as cs

from joblib import load

import src.methods.profiling as jp
from src.data_structures.join_discovery_methods import MinHashIndex

In [11]:
cfg = pl.Config()
cfg.set_fmt_str_lengths(30)

polars.config.Config

In [4]:
md_index_path = "data/metadata/_mdi/md_index_open_data_us.pickle"
md_index = load(md_index_path)

In [ ]:
real_table = pl.read_parquet("data/source_tables/schools-open_data.parquet")
real_table

col_to_embed,School Name,School Type,Total Parent Response Rate (%),Total Teacher Response Rate (%),Total Student Response Rate (%),target
str,str,str,i64,i64,f64,i8
"""01M015""","""P.S. 015 Rober…","""ES""",66,100,null,0
"""01M019""","""P.S. 019 Asher…","""ES""",96,97,null,0
"""01M020""","""P.S. 020 Anna …","""ES""",71,79,null,1
"""01M034""","""P.S. 034 Frank…","""ES/MS""",40,88,94.0,1
"""01M063""","""The STAR Acade…","""ES""",70,100,null,0
"""01M064""","""P.S. 064 Rober…","""ES""",78,85,null,0
"""01M110""","""P.S. 110 Flore…","""ES""",86,94,null,1
"""01M134""","""P.S. 134 Henri…","""ES""",29,74,null,1
"""01M137""","""P.S. 137 John …","""ES""",74,79,null,1


In [13]:
tgt_hash = "1f4cbb1f73a8cf5d61eb002a00deffee"
df_tgt = pl.read_parquet(md_index[tgt_hash]["full_path"])
df_tgt

c_0_column_1,c_1_column_2,c_2_column_3,c_3_column_4,c_4_column_5,c_5_column_6,c_6_column_7,c_7_column_8,c_8_column_9,c_9_column_10
str,str,str,str,str,str,str,str,str,str
"""DBN""","""School Name""","""School Type""","""Total Parent Response Rate (%…","""Total Teacher Response Rate (…","""Total Student Response Rate (…","""Total Academic Expectations S…","""Total Communication Score""","""Total Engagement Score""","""Total Safety and Respect Scor…"
"""01M015""","""P.S. 015 Roberto Clemente""","""ES""","""66""","""100""",null,"""8.3""","""8.3""","""8.1""","""8.4"""
"""01M019""","""P.S. 019 Asher Levy""","""ES""","""96""","""97""",null,"""8.7""","""8.8""","""8.5""","""8.6"""
"""01M020""","""P.S. 020 Anna Silver""","""ES""","""71""","""79""",null,"""7.9""","""7.6""","""7.7""","""8.1"""
"""01M034""","""P.S. 034 Franklin D. Roosevel…","""ES/MS""","""40""","""88""","""94.0""","""8.1""","""8.1""","""7.7""","""7.5"""
"""01M063""","""The STAR Academy – P.S.63""","""ES""","""70""","""100""",null,"""8.0""","""7.7""","""7.4""","""8.2"""
"""01M064""","""P.S. 064 Robert Simon""","""ES""","""78""","""85""",null,"""8.8""","""8.9""","""8.8""","""9.0"""
"""01M110""","""P.S. 110 Florence Nightingale…","""ES""","""86""","""94""",null,"""7.3""","""7.0""","""7.0""","""7.8"""
"""01M134""","""P.S. 134 Henrietta Szold""","""ES""","""29""","""74""",null,"""6.7""","""6.7""","""6.5""","""7.5"""


c_0_DBN,c_1_LOCATION NAME,c_2_LOCATION CATEGORY,c_3_ADMINISTRATIVE DISTRICT,c_4_NON-ELL STUDENTS With 2 or More Suspensions or Removals,c_5_ELL STUDENTS With 2 or More Suspensions or Removals,c_6_TOTAL STUDENTS With 2 or More Suspensions or Removals
str,str,str,f64,str,str,str
"""01M015""","""P.S. 015 Roberto Clemente""","""Elementary""",1.0,"""*""","""*""","""*"""
"""01M019""","""P.S. 019 Asher Levy""","""Elementary""",1.0,"""*""","""*""","""*"""
"""01M020""","""P.S. 020 Anna Silver""","""Elementary""",1.0,"""*""","""*""","""*"""
"""01M034""","""P.S. 034 Franklin D. Roosevel…","""K-8""",1.0,"""*""","""*""","""14"""
"""01M063""","""The STAR Academy - P.S.63""","""Elementary""",1.0,"""*""","""*""","""6"""
"""01M064""","""P.S. 064 Robert Simon""","""Elementary""",1.0,"""*""","""*""","""*"""
"""01M110""","""P.S. 110 Florence Nightingale…","""Elementary""",1.0,"""*""","""*""","""*"""
"""01M134""","""P.S. 134 Henrietta Szold""","""Elementary""",1.0,"""*""","""*""","""*"""
"""01M137""","""P.S. 137 John L. Bernstein""","""Elementary""",1.0,"""*""","""*""","""*"""


In [10]:
tgt2 = "687560c3e572d107352244dcda300402"
df_tgt = pl.read_parquet(md_index[tgt2]["full_path"])
df_tgt

c_0_DBN,c_1_LOCATION NAME,c_2_LOCATION CATEGORY,c_3_ADMINISTRATIVE DISTRICT,c_4_NON-ELL STUDENTS With 2 or More Suspensions or Removals,c_5_ELL STUDENTS With 2 or More Suspensions or Removals,c_6_TOTAL STUDENTS With 2 or More Suspensions or Removals
str,str,str,f64,str,str,str
"""01M015""","""P.S. 015 Rober…","""Elementary""",1.0,"""*""","""*""","""*"""
"""01M019""","""P.S. 019 Asher…","""Elementary""",1.0,"""*""","""*""","""*"""
"""01M020""","""P.S. 020 Anna …","""Elementary""",1.0,"""*""","""*""","""*"""
"""01M034""","""P.S. 034 Frank…","""K-8""",1.0,"""*""","""*""","""14"""
"""01M063""","""The STAR Acade…","""Elementary""",1.0,"""*""","""*""","""6"""
"""01M064""","""P.S. 064 Rober…","""Elementary""",1.0,"""*""","""*""","""*"""
"""01M110""","""P.S. 110 Flore…","""Elementary""",1.0,"""*""","""*""","""*"""
"""01M134""","""P.S. 134 Henri…","""Elementary""",1.0,"""*""","""*""","""*"""
"""01M137""","""P.S. 137 John …","""Elementary""",1.0,"""*""","""*""","""*"""
